# Quantum Embedding of Ising configurations

## Imports and random seed

In [1]:
import os
import random
import numpy as np
import glob
from qiskit import QuantumCircuit, Aer, IBMQ, transpile, QuantumRegister, ClassicalRegister
from qiskit.tools.jupyter import *
from qiskit.utils import QuantumInstance
from math import ceil, log2
from typing import Dict, List

from src.utils.train_test_split import train_test_split

In [2]:
seed = 6
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)

## Importing the data

In [3]:
# Size of the images, the number of qubits is 2 * ceil(log2(size_image)) + 1   (Height, Width and spin channel)
image_size = 16
# Ising infinite lattice size critical temperature
statistical_control_parameter = 2.2692 

with open('./data/ising/L={}/T={:.4f}.bin'.format(image_size, statistical_control_parameter), 'rb') as f:
    X = np.frombuffer(buffer=f.read(), dtype=np.int8, offset=0).reshape(-1, image_size, image_size)

idx = np.random.randint(X.shape[0])
image = X[idx]

In [4]:
image_size = 16
dataset_size = 5000

images = []
labels = []

for file_path in glob.glob('./data/ising/L=16/*'):
    with open(file_path, 'rb') as f:
        X = np.frombuffer(buffer=f.read(), dtype=np.int8, offset=0).reshape(-1, image_size, image_size)
        temperature = float(file_path.split('=')[-1].split('.bin')[0])
        y = np.full(shape=(X.shape[0],), fill_value=temperature)

        images.append(X)
        labels.append(y)

X = np.concatenate(images, axis=0)        
y = np.concatenate(labels, axis=0)   

# Permuting the dataset
idx = np.random.permutation(X.shape[0])
X = X[idx][:dataset_size]
y = y[idx][:dataset_size]

# Splitting train set into train and validation set
X_train, y_train, X_test, y_test = train_test_split(X, y, validation_fraction=0.2)

## NEQR Quantum encoding

In [5]:
def binary_formatting(
    digit: int,
    n_bits: int,
    reverse: bool = False,
) -> str:
    r"""
    Args:
        digit (int): number whose binary representation we are computing
        n_bits (int): number of bits used in the binary representation (to handle left trailing zeros)
        reverse (bool): optionally return the binary representation of digit in reverse order (for qiskit convention)
    Returns:
        (str): binary representation of digit
    """

    binary = format(digit, '0{}b'.format(n_bits))

    if reverse:
        binary = binary[::-1]

    return binary

In [6]:
side_qubits = ceil(log2(image_size))
image_qubits = 2 * side_qubits
total_qubits = image_qubits + 1

We will assign |0> to a -1 spin and |1> to a +1 spin.

In [7]:
def image_to_circuit(
    image: np.ndarray,
    measure: bool = False,
) -> QuantumCircuit:

    image_size = image.shape[0]
    side_qubits = ceil(log2(image_size))
    image_qubits = 2 * side_qubits
    total_qubits = image_qubits + 1

    image_register = QuantumRegister(image_qubits, 'position')
    spin_register = QuantumRegister(1,'spin')

    if measure:
        classical_register = ClassicalRegister(image_qubits + 1, 'classical register')
        qc = QuantumCircuit(spin_register, image_register, classical_register)

    else:
        qc = QuantumCircuit(spin_register, image_register)

    qc.i(0)
    for i in range(1, image_register.size + 1):
        qc.h(i)

    qc.barrier()

    for i in range(image_size):
        for j in range(image_size):

            if image[i, j] == 1:

                binarized_i = binary_formatting(digit=i, n_bits=side_qubits, reverse=False)
                binarized_j = binary_formatting(digit=j, n_bits=side_qubits, reverse=False)

                flip_idx = []

                for ii in range(side_qubits):
                    if binarized_i[ii] == '1':
                        flip_idx.append(ii+1)
                        qc.x(ii+1)

                for jj in range(side_qubits):
                    if binarized_j[jj] == '1':
                        flip_idx.append(jj+side_qubits+1)
                        qc.x(jj+side_qubits+1)

                qc.mcx(list(range(1, image_qubits+1)), 0, mode='noancilla')

                for q in flip_idx:
                    qc.x(q)

                qc.barrier() 

    if measure:
        qc.measure(range(total_qubits), range(total_qubits))

    return qc

In [ ]:
qc = image_to_circuit(image=image, measure=False)
qc.draw(output='mpl')
print('Circuit depth: ', qc.decompose().depth())
print('Circuit size: ', qc.decompose().size())
qc.decompose().count_ops()

In [8]:
def counts_to_statevector(
    counts: Dict[str, int],
    num_qubits: int,
) -> np.ndarray:
    
    statevector = np.zeros(shape=(2**num_qubits,))
    for key, value in counts.items():
        statevector[int(key, 2)] = value
    statevector = statevector / np.linalg.norm(statevector)
    return statevector

def statevector_to_densitymatrix(
    statevector: np.ndarray
) -> np.ndarray:
    r"""
    Args:
        statevector (np.ndarray): statevector corresponding to a pure state
    Return:
        (np.ndarray): density matrix corresponding to the pure state statevector
    """

    density_matrix = np.outer(statevector, statevector)
    return density_matrix

def images_to_quantumstate(
    image_batch: np.ndarray,
    from_counts: bool = False,
    to_real: bool = False,
    output_densitymatrices: bool = False
) -> List[np.ndarray]:
    r"""
    Args:
        image_batch (np.ndarray): batch of images whose quantum representation we are computing
        from_counts (np.ndarray): whether or not to reconstruct the output statevector statistically from counts or not
        to_real (bool): whether or not to cast the complex components of statevector to real ones
        output_densitymatrices (bool): whether to output the density matrices of the statevectors
    Return:
        (List[np.ndarray]): batch of density matrix representation of the images
    """

    backend = Aer.get_backend('statevector_simulator')
    bounds = []
    num_qubits = 2 * ceil(log2(image_batch.shape[-1])) + 1

    for i in range(image_batch.shape[0]):
        q = image_to_circuit(image=image_batch[i], measure=False)
        bounds.append(q)
        
    qc = transpile(bounds, backend)

    if from_counts:
        qi = QuantumInstance(backend, seed_transpiler=42, seed_simulator=42, shots=2048)
        result = qi.execute(circuits=qc, had_transpiled=True)
        counts = result.get_counts() # It seems we don't even have to explicitely add the measurement explicitely to the circuit

        # Handle the case of a batch containing a single image
        if type(counts) == dict:
            counts = [counts]

        if to_real:
            import warnings
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                statevector = [counts_to_statevector(counts[ind], num_qubits).astype('float32') for ind in range(len(counts))]
        else:
            statevector = [counts_to_statevector(counts[ind], num_qubits) for ind in range(len(counts))]

    else:
        qi = QuantumInstance(backend, seed_transpiler=42, seed_simulator=42)
        result = qi.execute(circuits=qc, had_transpiled=True)

        if to_real:
            import warnings
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                statevector = [result.get_statevector(i).astype('float32') for ind in range(len(bounds))]
        else:
            statevector = [result.get_statevector(i) for i in range(len(bounds))]

    statevector = np.stack(statevector)

    if output_densitymatrices:
        densitymatrices = np.einsum('ab,ac->abc', statevector, statevector)
        return densitymatrices

    else:
        return statevector

In [9]:
X_train.shape

(4000, 16, 16)

In [10]:
batch_size = 200

idx = 1
for i in range(0, X_train.shape[0], batch_size):
    statevector_train = images_to_quantumstate(image_batch=X_train[i: i+batch_size], from_counts=False, to_real=True, output_densitymatrices=False)
    with open('./data/ising/preprocessed/statevector_L=16_train_{}.npy'.format(idx), 'wb') as f:
        np.save(f, statevector_train)
    idx += 1

idx = 1
for i in range(0, X_test.shape[0], batch_size):
    statevector_test = images_to_quantumstate(image_batch=X_test[i: i+batch_size], from_counts=False, to_real=True, output_densitymatrices=False)
    with open('./data/ising/preprocessed/statevector_L=16_test_{}.npy'.format(idx), 'wb') as f:
        np.save(f, statevector_test)
    idx += 1

with open('./data/ising/preprocessed/label_L=16_train.npy', 'wb') as f:
    np.save(f, y_train)
with open('./data/ising/preprocessed/label_L=16_test.npy', 'wb') as f:
    np.save(f, y_test)

## Temperature prediction